In [38]:
# 1.) Download the PDB chain sequences (FASTA format from RCSB) via the HTTPS mirror,
#      then keep only the first 15 000 entries.

import requests

# Use the “files.wwpdb.org” HTTPS mirror instead of FTP
pdb_url = "https://files.wwpdb.org/pub/pdb/derived_data/pdb_seqres.txt"
try:
    resp = requests.get(pdb_url, timeout=60)
    resp.raise_for_status()
    text = resp.text.strip()
    if not text.startswith(">"):
        raise RuntimeError("Downloaded content does not look like FASTA.")
except Exception as e:
    raise RuntimeError(f"Failed to download PDB chain sequences: {e}")

# Write the complete dump to a temporary file
with open("pdb_chains.fasta", "w", encoding="utf-8") as f:
    f.write(text + "\n")

# ─── Split the full FASTA into individual (header, sequence) tuples ─────────────
def split_fasta(filepath):
    sequences = []
    with open(filepath, "r") as f:
        header = None
        seq_lines = []
        for line in f:
            line = line.rstrip()
            if line.startswith(">"):
                if header is not None:
                    sequences.append((header, "".join(seq_lines)))
                header = line
                seq_lines = []
            else:
                seq_lines.append(line)
        # Add the final sequence
        if header is not None:
            sequences.append((header, "".join(seq_lines)))
    return sequences

all_chains = split_fasta("pdb_chains.fasta")

# ─── Keep exactly the first 15 000 chains ─────────────────────────────────────────
subset = all_chains[:15000]

# ─── Write those 15 000 chains back to “pdb_chains.fasta” ───────────────────────
with open("pdb_chains.fasta", "w", encoding="utf-8") as f:
    for header, seq in subset:
        f.write(f"{header}\n")
        f.write(f"{seq}\n")

print(f"✔ Extracted {len(subset)} chains → 'pdb_chains.fasta'")


✔ Extracted 15000 chains → 'pdb_chains.fasta'


In [34]:
# 2.) Use DisProt’s search endpoint with format=fasta
import requests
import os

url = "https://disprot.org/api/search?format=fasta&limit=10000"
try:
    resp = requests.get(url, timeout=15)
    resp.raise_for_status()
except Exception as e:
    raise RuntimeError(f"Failed to GET DisProt FASTA via API: {e}")

text = resp.text.strip()

# 2.2) Quick sanity check: FASTA must start with '>', not '<'
if not text.startswith(">"):
    raise RuntimeError(
        "Downloaded content does not look like FASTA. "
        "If it begins with '<', you're still hitting an HTML page instead of raw FASTA."
    )

# 2.3) Write the 100 DisProt entries to a file
with open("disprot_13000.fasta", "w") as f:
    f.write(text + "\n")

print("✔ Successfully fetched 100 DisProt sequences in FASTA format → 'disprot_1000.fasta'")


✔ Successfully fetched 100 DisProt sequences in FASTA format → 'disprot_1000.fasta'


In [14]:
# 2.1) Collect more data

import requests
import time

# ─── PARAMETERS ─────────────────────────────────────────────────────────────
TOTAL_DESIRED = 25_000   # how many DisProt sequences we want total
PER_PAGE      = 100      # DisProt’s hard cap per request
OUTPUT_FILE   = "disprot_13000.fasta"

accum_seqs = []
offset     = 0

while len(accum_seqs) < TOTAL_DESIRED:
    url = f"https://disprot.org/api/search?format=fasta&limit={PER_PAGE}&offset={offset}"
    try:
        resp = requests.get(url, timeout=30)
        resp.raise_for_status()
    except Exception as e:
        raise RuntimeError(f"Failed to GET DisProt FASTA (offset={offset}): {e}")

    block = resp.text.strip()
    if not block.startswith(">"):
        raise RuntimeError(
            "Downloaded content does not look like FASTA. "
            "If it begins with '<', you're still hitting an HTML page."
        )

    # Parse out this page’s FASTA sequences (collecting only the raw sequences, not full headers):
    raw_lines = block.splitlines()
    header = None
    seq_buf = ""
    this_page_seqs = []
    for line in raw_lines:
        if line.startswith(">"):
            if header is not None and seq_buf:
                this_page_seqs.append(seq_buf)
            header = line
            seq_buf = ""
        else:
            seq_buf += line.strip()
    if header is not None and seq_buf:
        this_page_seqs.append(seq_buf)

    if not this_page_seqs:
        # No more sequences returned → break out early
        break

    accum_seqs.extend(this_page_seqs)
    offset += PER_PAGE

    # Sleep briefly (so we don’t hammer the server)
    time.sleep(0.4)

# Trim in case we overshot
accum_seqs = accum_seqs[:TOTAL_DESIRED]

# Write out ~25k sequences in FASTA format (with minimal headers)
with open(OUTPUT_FILE, "w") as f:
    for i, seq in enumerate(accum_seqs):
        f.write(f">disprot_sequence_{i+1}\n")
        f.write(seq + "\n")

print(f"✔ Fetched {len(accum_seqs)} DisProt sequences → '{OUTPUT_FILE}'")


✔ Fetched 25000 DisProt sequences → 'disprot_13000.fasta'


In [15]:
# 2.2) Verify Downloaded Sequences
with open("disprot_13000.fasta") as f:
    for _ in range(5):
        print(f.readline().rstrip())


>disprot_sequence_1
EHVIEMDVTSENGQRALKEQSSKAKIVKNRWGRNVVQISNT
>disprot_sequence_2
VYRNSRAQGGG
>disprot_sequence_3


# Constraint Based - (Concept Model)

In [54]:
# 3.) Seven‐Feature Threshold‐Based Fold/Disorder Classifier


import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

# ─── (A) Build aa_properties ─────────────────────────────
kd_hydro = {
    'A':  1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C':  2.5,
    'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I':  4.5,
    'L':  3.8, 'K': -3.9, 'M':  1.9, 'F':  2.8, 'P': -1.6,
    'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V':  4.2
}
charge = {
    'A':  0, 'R':  1, 'N':  0, 'D': -1, 'C':  0,
    'Q':  0, 'E': -1, 'G':  0, 'H':  0, 'I':  0,
    'L':  0, 'K':  1, 'M':  0, 'F':  0, 'P':  0,
    'S':  0, 'T':  0, 'W':  0, 'Y':  0, 'V':  0
}
h_donors = {'A':0,'R':2,'N':2,'D':0,'C':0,'Q':2,'E':0,'G':0,'H':1,'I':0,
            'L':0,'K':1,'M':0,'F':0,'P':0,'S':1,'T':1,'W':1,'Y':1,'V':0}
h_acceptors = {'A':0,'R':0,'N':2,'D':2,'C':1,'Q':2,'E':2,'G':0,'H':1,'I':0,
               'L':0,'K':0,'M':0,'F':0,'P':0,'S':1,'T':1,'W':0,'Y':1,'V':0}
flexibility = {
    'A': 0.357, 'R': 0.529, 'N': 0.463, 'D': 0.511, 'C': 0.346,
    'Q': 0.493, 'E': 0.497, 'G': 0.544, 'H': 0.323, 'I': 0.462,
    'L': 0.365, 'K': 0.466, 'M': 0.295, 'F': 0.314, 'P': 0.509,
    'S': 0.507, 'T': 0.444, 'W': 0.305, 'Y': 0.420, 'V': 0.386
}
sidechain_volume = {
    'A':  88.6, 'R': 173.4, 'N': 114.1, 'D': 111.1, 'C': 108.5,
    'Q': 143.8, 'E': 138.4, 'G':  60.1, 'H': 153.2, 'I': 166.7,
    'L': 166.7, 'K': 168.6, 'M': 162.9, 'F': 189.9, 'P': 112.7,
    'S':  89.0, 'T': 116.1, 'W': 227.8, 'Y': 193.6, 'V': 140.0
}
polarity = {
    'A':  8.1, 'R': 10.5, 'N': 11.6, 'D': 13.0, 'C':  5.5,
    'Q': 10.5, 'E': 12.3, 'G':  9.0, 'H': 10.4, 'I':  5.2,
    'L':  4.9, 'K': 11.3, 'M':  5.7, 'F':  5.2, 'P':  8.0,
    'S':  9.2, 'T':  8.6, 'W':  5.4, 'Y':  6.2, 'V':  5.9
}
choufa_helix = {
    'A': 1.45, 'R': 0.79, 'N': 0.73, 'D': 1.01, 'C': 0.77,
    'Q': 1.17, 'E': 1.51, 'G': 0.53, 'H': 1.00, 'I': 1.08,
    'L': 1.34, 'K': 1.07, 'M': 1.20, 'F': 1.12, 'P': 0.59,
    'S': 0.79, 'T': 0.82, 'W': 1.14, 'Y': 0.61, 'V': 1.06
}
choufa_sheet = {
    'A': 0.97, 'R': 0.90, 'N': 0.65, 'D': 0.54, 'C': 1.30,
    'Q': 1.23, 'E': 0.37, 'G': 0.75, 'H': 0.87, 'I': 1.60,
    'L': 1.22, 'K': 0.74, 'M': 1.67, 'F': 1.28, 'P': 0.62,
    'S': 0.72, 'T': 1.20, 'W': 1.19, 'Y': 1.29, 'V': 1.70
}
rel_ASA = {
    'A': 0.74, 'R': 1.48, 'N': 1.14, 'D': 1.23, 'C': 0.86,
    'Q': 1.36, 'E': 1.26, 'G': 1.00, 'H': 0.91, 'I': 0.59,
    'L': 0.61, 'K': 1.29, 'M': 0.64, 'F': 0.65, 'P': 0.71,
    'S': 1.42, 'T': 1.20, 'W': 0.55, 'Y': 0.63, 'V': 0.54
}
beta_branched = {aa: (1 if aa in ('V','I','T') else 0) for aa in kd_hydro.keys()}

# Build aa_properties dictionary (12 dimensions per residue)
aa_properties = {}
canonical_set = set(kd_hydro.keys())
for aa in canonical_set:
    hydro_norm  = (kd_hydro[aa] + 4.5) / 9.0
    volume_norm = sidechain_volume[aa] / 227.8
    pol_norm    = (polarity[aa] - 4.9) / (13.0 - 4.9)
    helix_norm  = choufa_helix[aa] / 1.51
    sheet_norm  = choufa_sheet[aa] / 1.70
    asa_norm    = (rel_ASA[aa] - 0.54) / (1.48 - 0.54)
    aromatic    = 1 if aa in ('F','Y','W') else 0

    aa_properties[aa] = [
        hydro_norm,          # [0]
        charge[aa],          # [1]
        h_donors[aa],        # [2]
        h_acceptors[aa],     # [3]
        flexibility[aa],     # [4]
        volume_norm,         # [5]
        pol_norm,            # [6]
        aromatic,            # [7]
        helix_norm,          # [8]
        sheet_norm,          # [9]
        asa_norm,            # [10]
        beta_branched[aa]    # [11]
    ]

# ─── (B) Load FASTA sequences ─────────────────────────────────────────────────
def load_fasta(filepath, filter_non_canonical=False):
    seqs = []
    with open(filepath) as f:
        header = None
        seq = ""
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if header is not None and seq:
                    if (not filter_non_canonical) or (set(seq) <= canonical_set):
                        seqs.append(seq)
                header = line
                seq = ""
            else:
                seq += line
        if header is not None and seq:
            if (not filter_non_canonical) or (set(seq) <= canonical_set):
                seqs.append(seq)
    return seqs

pdb_seqs    = load_fasta("pdb_chains.fasta",   filter_non_canonical=False)   # 70 PDB chains
disprot_seqs = load_fasta("disprot_13000.fasta", filter_non_canonical=False)  # 100 DisProt

# ─── (C) Compute each chain’s 7 global features ────────────────────────────────
def compute_global_features(sequence):
    props = []
    for aa in sequence:
        if aa in aa_properties:
            v = aa_properties[aa]
            props.append([
                v[0],               # hydrophobicity_norm
                v[1],               # charge
                v[2] + v[3],        # h_dh_a
                v[4] / 0.544,       # norm_flex (raw_flex/0.544)
                v[6],               # pol_norm
                v[7] + v[8],        # arom_plus_helix
                v[10]               # asa_norm
            ])
    if not props:
        return np.zeros(7)
    return np.mean(np.vstack(props), axis=0)

all_features = []
all_labels   = []

for seq in pdb_seqs:
    all_features.append(compute_global_features(seq))
    all_labels.append(1)   # 1 = folded (PDB)
for seq in disprot_seqs:
    all_features.append(compute_global_features(seq))
    all_labels.append(0)   # 0 = disordered (DisProt)

df_feat = pd.DataFrame(
    all_features,
    columns=[
        "hydro_norm",
        "charge",
        "h_dh_a",
        "norm_flex",
        "pol_norm",
        "arom_plus_helix",
        "asa_norm"
    ]
)
df_feat["label"] = all_labels

# ─── (D) Compute midpoint thresholds (mean of PDB vs. mean of DisProt) ───────
means = df_feat.groupby("label").mean().rename(index={0:"DisProt", 1:"PDB"})
midpoints = {col: (means.loc["PDB", col] + means.loc["DisProt", col]) / 2
             for col in df_feat.columns[:-1]}

print("Global Feature Means (DisProt vs. PDB):\n")
print(means, "\n")
print("Chosen Midpoint Thresholds:\n")
for feat, t in midpoints.items():
    print(f"  {feat:18s} = {t:.3f}")
print()

# ─── (E) Count how many of the 7 conditions each chain satisfies ───────────────
def count_conditions(row):
    c1 = row["hydro_norm"]          >= midpoints["hydro_norm"]
    c2 = abs(row["charge"])         <= abs(midpoints["charge"])
    c3 = row["h_dh_a"]              <= midpoints["h_dh_a"]
    c4 = row["norm_flex"]           <= midpoints["norm_flex"]
    c5 = row["pol_norm"]            <= midpoints["pol_norm"]
    c6 = row["arom_plus_helix"]     >= midpoints["arom_plus_helix"]
    c7 = row["asa_norm"]            <= midpoints["asa_norm"]
    return sum([c1, c2, c3, c4, c5, c6, c7])

df_feat["conditions_met"] = df_feat.apply(count_conditions, axis=1)

# Show the distribution of “conditions_met” separately for PDB vs. DisProt
dist = df_feat.groupby("label")["conditions_met"] \
              .value_counts() \
              .unstack(fill_value=0) \
              .rename(index={0:"DisProt", 1:"PDB"})

pd.set_option("display.max_columns", None)
print("Distribution of ‘conditions_met’ by Label:\n")
print(dist, "\n")

# ─── (F) For each k=1…7, classify “folded if conditions_met ≥ k” ─────────────
results = []
for k in range(1, 8):
    preds = (df_feat["conditions_met"] >= k).astype(int)
    tp = ((preds == 1) & (df_feat["label"] == 1)).sum()
    fn = ((preds == 0) & (df_feat["label"] == 1)).sum()
    tn = ((preds == 0) & (df_feat["label"] == 0)).sum()
    fp = ((preds == 1) & (df_feat["label"] == 0)).sum()
    acc = (tp + tn) / len(df_feat)
    results.append({
        "k (min # of features)": k,
        "TP": tp,
        "FN": fn,
        "TN": tn,
        "FP": fp,
        "Accuracy": f"{acc:.2%}"
    })

df_results = pd.DataFrame(results)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
print("Performance as we vary k = minimum # of satisfied conditions:\n")
print(df_results.to_string(index=False))


Global Feature Means (DisProt vs. PDB):

         hydro_norm    charge    h_dh_a  norm_flex  pol_norm  arom_plus_helix  \
label                                                                           
DisProt    0.401170 -0.023269  1.255558   0.837162  0.511919         0.718951   
PDB        0.475469 -0.008521  1.071916   0.806189  0.436776         0.734963   

         asa_norm  
label              
DisProt  0.519567  
PDB      0.444721   

Chosen Midpoint Thresholds:

  hydro_norm         = 0.438
  charge             = -0.016
  h_dh_a             = 1.164
  norm_flex          = 0.822
  pol_norm           = 0.474
  arom_plus_helix    = 0.727
  asa_norm           = 0.482

Distribution of ‘conditions_met’ by Label:

conditions_met     0     1     2     3     4     5     6     7
label                                                         
DisProt         3116  3265  1762  1342  1204   980   922   196
PDB              156   492   812   925  1525  2832  5579  2679 

Performance as we va

# Derive Coefficients w/ LR

In [55]:
# 3.1.) Logistic Regression–Derived Seven‐Feature Classifier
 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# (1) Split the same feature matrix and label vector into train/test
X = df_feat.drop(columns=["label"])
y = df_feat["label"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# (2) Fit the logistic model (with class_weight='balanced'):
clf = LogisticRegression(
    penalty='l2',
    C=1.0,
    class_weight='balanced',
    solver='lbfgs',
    max_iter=200,
    random_state=42
).fit(X_train, y_train)

# (3) After fitting, these attributes hold exactly the numbers we used:
print(clf.coef_.flatten())   # → [ 9.149,  3.051,  2.034, -7.553, -6.521,  8.728, -7.629 ]
print(clf.intercept_)        # → [0.131]


[ 1.86285759  1.57922578  1.96082866 -0.99457729 -1.5543599  -2.86804225
 -1.22492812  0.79904486]
[-1.84274409]


# 4.) Trained

In [56]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# ─── (A) Build aa_properties ───────────────────────────────────────────────────
# (Amino acid properties dictionaries remain the same as in your original code)
kd_hydro = {
    'A':  1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C':  2.5,
    'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I':  4.5,
    'L':  3.8, 'K': -3.9, 'M':  1.9, 'F':  2.8, 'P': -1.6,
    'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V':  4.2
}
charge = {
    'A':  0, 'R':  1, 'N':  0, 'D': -1, 'C':  0,
    'Q':  0, 'E': -1, 'G':  0, 'H':  0, 'I':  0,
    'L':  0, 'K':  1, 'M':  0, 'F':  0, 'P':  0,
    'S':  0, 'T':  0, 'W':  0, 'Y':  0, 'V':  0
}
h_donors = {'A':0,'R':2,'N':2,'D':0,'C':0,'Q':2,'E':0,'G':0,'H':1,'I':0,
            'L':0,'K':1,'M':0,'F':0,'P':0,'S':1,'T':1,'W':1,'Y':1,'V':0}
h_acceptors = {'A':0,'R':0,'N':2,'D':2,'C':1,'Q':2,'E':2,'G':0,'H':1,'I':0,
               'L':0,'K':0,'M':0,'F':0,'P':0,'S':1,'T':1,'W':0,'Y':1,'V':0}
flexibility = {
    'A': 0.357, 'R': 0.529, 'N': 0.463, 'D': 0.511, 'C': 0.346,
    'Q': 0.493, 'E': 0.497, 'G': 0.544, 'H': 0.323, 'I': 0.462,
    'L': 0.365, 'K': 0.466, 'M': 0.295, 'F': 0.314, 'P': 0.509,
    'S': 0.507, 'T': 0.444, 'W': 0.305, 'Y': 0.420, 'V': 0.386
}
sidechain_volume = {
    'A':  88.6, 'R': 173.4, 'N': 114.1, 'D': 111.1, 'C': 108.5,
    'Q': 143.8, 'E': 138.4, 'G':  60.1, 'H': 153.2, 'I': 166.7,
    'L': 166.7, 'K': 168.6, 'M': 162.9, 'F': 189.9, 'P': 112.7,
    'S':  89.0, 'T': 116.1, 'W': 227.8, 'Y': 193.6, 'V': 140.0
}
polarity = {
    'A':  8.1, 'R': 10.5, 'N': 11.6, 'D': 13.0, 'C':  5.5,
    'Q': 10.5, 'E': 12.3, 'G':  9.0, 'H': 10.4, 'I':  5.2,
    'L':  4.9, 'K': 11.3, 'M':  5.7, 'F':  5.2, 'P':  8.0,
    'S':  9.2, 'T':  8.6, 'W':  5.4, 'Y':  6.2, 'V':  5.9
}
choufa_helix = {
    'A': 1.45, 'R': 0.79, 'N': 0.73, 'D': 1.01, 'C': 0.77,
    'Q': 1.17, 'E': 1.51, 'G': 0.53, 'H': 1.00, 'I': 1.08,
    'L': 1.34, 'K': 1.07, 'M': 1.20, 'F': 1.12, 'P': 0.59,
    'S': 0.79, 'T': 0.82, 'W': 1.14, 'Y': 0.61, 'V': 1.06
}
choufa_sheet = {
    'A': 0.97, 'R': 0.90, 'N': 0.65, 'D': 0.54, 'C': 1.30,
    'Q': 1.23, 'E': 0.37, 'G': 0.75, 'H': 0.87, 'I': 1.60,
    'L': 1.22, 'K': 0.74, 'M': 1.67, 'F': 1.28, 'P': 0.62,
    'S': 0.72, 'T': 1.20, 'W': 1.19, 'Y': 1.29, 'V': 1.70
}
rel_ASA = {
    'A': 0.74, 'R': 1.48, 'N': 1.14, 'D': 1.23, 'C': 0.86,
    'Q': 1.36, 'E': 1.26, 'G': 1.00, 'H': 0.91, 'I': 0.59,
    'L': 0.61, 'K': 1.29, 'M': 0.64, 'F': 0.65, 'P': 0.71,
    'S': 1.42, 'T': 1.20, 'W': 0.55, 'Y': 0.63, 'V': 0.54
}
beta_branched = {aa: (1 if aa in ('V','I','T') else 0) for aa in kd_hydro.keys()}

aa_properties = {}
canonical_set = set(kd_hydro.keys())
for aa in canonical_set:
    hydro_norm  = (kd_hydro[aa] + 4.5) / 9.0
    volume_norm = sidechain_volume[aa] / 227.8
    pol_norm    = (polarity[aa] - 4.9) / (13.0 - 4.9)
    helix_norm  = choufa_helix[aa] / 1.51
    sheet_norm  = choufa_sheet[aa] / 1.70
    asa_norm    = (rel_ASA[aa] - 0.54) / (1.48 - 0.54)
    aromatic    = 1 if aa in ('F','Y','W') else 0

    aa_properties[aa] = [
        hydro_norm,          # [0]
        charge[aa],          # [1]
        h_donors[aa],        # [2]
        h_acceptors[aa],     # [3]
        flexibility[aa],     # [4]
        volume_norm,         # [5]
        pol_norm,            # [6]
        aromatic,            # [7]
        helix_norm,          # [8]
        sheet_norm,          # [9]
        asa_norm,            # [10]
        beta_branched[aa]    # [11]
    ]

# ─── (B) Load FASTA sequences ─────────────────────────────────────────────────
def load_fasta(filepath, filter_non_canonical=False):
    seqs = []
    try:
        with open(filepath) as f:
            header = None
            seq = ""
            for line in f:
                line = line.strip()
                if line.startswith(">"):
                    if header is not None and seq:
                        if (not filter_non_canonical) or (set(seq) <= canonical_set):
                            seqs.append(seq)
                    header = line
                    seq = ""
                else:
                    seq += line
            if header is not None and seq: # Add the last sequence
                if (not filter_non_canonical) or (set(seq) <= canonical_set):
                    seqs.append(seq)
    except FileNotFoundError:
        print(f"Warning: File not found {filepath}. Returning empty list.")
    return seqs

# Ensure these files exist in the same directory as your script,
# or provide full paths.
pdb_seqs    = load_fasta("pdb_chains.fasta",   filter_non_canonical=False)
disprot_seqs = load_fasta("disprot_13000.fasta", filter_non_canonical=False)

print(f"Loaded {len(pdb_seqs)} PDB sequences.")
print(f"Loaded {len(disprot_seqs)} DisProt sequences.")

if not pdb_seqs and not disprot_seqs:
    print("Error: No sequences loaded. Exiting.")
    exit()
elif not pdb_seqs:
    print("Warning: No PDB sequences loaded. Classifier might not be meaningful.")
elif not disprot_seqs:
    print("Warning: No DisProt sequences loaded. Classifier might not be meaningful.")


# ─── (C) Compute each chain’s 7 global features ────────────────────────────────
def compute_global_features(sequence):
    props = []
    # Filter out non-canonical amino acids from the sequence before processing
    # to avoid KeyError if a non-canonical AA is encountered.
    # However, the current aa_properties dictionary is built only from canonical_set,
    # so this check is implicitly handled by `if aa in aa_properties:`.
    # If sequences can contain 'X', 'U', 'O', etc., more robust filtering might be needed
    # or aa_properties expanded.
    valid_aas_in_sequence = 0
    for aa in sequence:
        if aa in aa_properties:
            v = aa_properties[aa]
            props.append([
                v[0],               # hydrophobicity_norm
                v[1],               # charge
                v[2] + v[3],        # h_dh_a (sum of H-bond donors and acceptors)
                v[4] / 0.544,       # norm_flex (flexibility normalized by G's flexibility)
                v[6],               # pol_norm (normalized polarity)
                v[7] + v[8],        # arom_plus_helix (sum of aromatic indicator and helix propensity)
                v[10]               # asa_norm (normalized relative solvent accessibility)
            ])
            valid_aas_in_sequence +=1
    
    if not props or valid_aas_in_sequence == 0: # Handle empty sequences or sequences with no canonical AAs
        return np.zeros(7) # Return a vector of zeros if no properties could be computed
    return np.mean(np.vstack(props), axis=0)

all_features_list = [] # Changed name to avoid conflict with pandas df
all_labels_list   = [] # Changed name

for seq in pdb_seqs:
    # Skip empty sequences if any
    if seq: 
        all_features_list.append(compute_global_features(seq))
        all_labels_list.append(1)   # 1 = folded (PDB)

for seq in disprot_seqs:
    # Skip empty sequences if any
    if seq:
        all_features_list.append(compute_global_features(seq))
        all_labels_list.append(0)   # 0 = disordered (DisProt)

if not all_features_list:
    print("Error: No features could be computed from the loaded sequences. Exiting.")
    exit()

df_all_data = pd.DataFrame(
    all_features_list,
    columns=[
        "hydro_norm",
        "charge",
        "h_dh_a",
        "norm_flex",
        "pol_norm",
        "arom_plus_helix",
        "asa_norm"
    ])
df_all_data["label"] = all_labels_list

# ─── (D) Split data into Training and Testing sets ────────────────────────────
# Ensure there's enough data to split, especially for stratification
if df_all_data['label'].nunique() < 2:
    print("Error: Need at least two classes for stratification. Exiting.")
    # Or handle differently, e.g., proceed without stratification if only one class loaded
    # For now, exiting as classification is not meaningful.
    if not df_all_data.empty:
         print(df_all_data['label'].value_counts())
    exit()
    
# Check if each class has enough members for the split
min_class_count = df_all_data['label'].value_counts().min()
if min_class_count < 2 and len(df_all_data) > 1 : # n_splits for StratifiedKFold is 2 by default for test_size > 0
    print(f"Warning: The smallest class has only {min_class_count} member(s), which might be too few for stratified splitting depending on test_size.")
    # Proceeding with splitting, but be aware of potential issues if test_size is too large relative to min_class_count
    # If min_class_count is 1, stratification will fail.
    if min_class_count == 1:
        print("Error: Smallest class has only 1 member. Stratification requires at least 2 members per class. Consider non-stratified split or getting more data.")
        # As a fallback, could do a non-stratified split if user desires, but for now, exiting.
        exit()


# Use a test_size, e.g., 0.2 for 20% test data
# random_state for reproducibility
# stratify by labels to maintain class proportions in train and test sets
try:
    X_train, X_test, y_train, y_test = train_test_split(
        df_all_data.drop(columns=["label"]),
        df_all_data["label"],
        test_size=0.2,
        random_state=42,
        stratify=df_all_data["label"] # Stratify if both classes are present
    )
except ValueError as e:
    print(f"Error during train_test_split, possibly due to insufficient samples in a class for stratification: {e}")
    print("Consider using a smaller test_size or ensuring more samples per class.")
    # Fallback: try without stratification if error is due to it and min_class_count was an issue
    # For now, exiting.
    exit()


df_train = X_train.copy()
df_train["label"] = y_train

df_test = X_test.copy()
df_test["label"] = y_test

print(f"\nTraining set size: {len(df_train)}")
print(f"Testing set size: {len(df_test)}")
print(f"Training set PDB (1) count: {y_train.sum()}, DisProt (0) count: {len(y_train) - y_train.sum()}")
print(f"Testing set PDB (1) count: {y_test.sum()}, DisProt (0) count: {len(y_test) - y_test.sum()}")


# ─── (E) Compute midpoint thresholds (mean of PDB vs. mean of DisProt) ON TRAINING DATA ONLY ───
# Check if both labels are present in the training set after split
if df_train["label"].nunique() < 2:
    print("\nError: Training set does not contain both classes after split. Cannot compute midpoints.")
    # This can happen if one class had very few samples and all went to the test set (unlikely with stratification but possible with tiny datasets)
    # Or if the initial dataset was highly imbalanced and very small.
    print(df_train["label"].value_counts())
    midpoints = {col: 0.5 for col in X_train.columns} # Default midpoints if calculation fails
    print("Warning: Using default midpoints (0.5) as training data lacks class diversity.")
else:
    train_means = df_train.groupby("label").mean().rename(index={0:"DisProt", 1:"PDB"})
    # Ensure both "PDB" and "DisProt" indices exist in train_means before calculating midpoints
    if "PDB" not in train_means.index or "DisProt" not in train_means.index:
        print("\nError: Could not find means for both PDB and DisProt in the training set.")
        print(train_means)
        midpoints = {col: 0.5 for col in X_train.columns} # Default midpoints
        print("Warning: Using default midpoints (0.5) as PDB/DisProt means are missing in training data.")
    else:
        midpoints = {col: (train_means.loc["PDB", col] + train_means.loc["DisProt", col]) / 2
                     for col in X_train.columns} # Iterate over X_train.columns (feature columns)

        print("\nGlobal Feature Means (DisProt vs. PDB) from TRAINING DATA:\n")
        print(train_means, "\n")
        print("Chosen Midpoint Thresholds (from TRAINING DATA):\n")
        for feat, t in midpoints.items():
            print(f"  {feat:18s} = {t:.3f}")
        print()

# ─── (F) Count how many of the 7 conditions each chain in the TEST SET satisfies ───────────────
# This function uses the 'midpoints' dictionary calculated from the training data
def count_conditions_test(row, midpoints_dict):
    c1 = row["hydro_norm"]          >= midpoints_dict["hydro_norm"]
    c2 = abs(row["charge"])         <= abs(midpoints_dict["charge"]) # abs for midpoint too, if it could be negative
    c3 = row["h_dh_a"]              <= midpoints_dict["h_dh_a"]
    c4 = row["norm_flex"]           <= midpoints_dict["norm_flex"]
    c5 = row["pol_norm"]            <= midpoints_dict["pol_norm"]
    c6 = row["arom_plus_helix"]     >= midpoints_dict["arom_plus_helix"]
    c7 = row["asa_norm"]            <= midpoints_dict["asa_norm"]
    return sum([c1, c2, c3, c4, c5, c6, c7])

# Apply to the test set
df_test["conditions_met"] = df_test.apply(lambda row: count_conditions_test(row, midpoints), axis=1)


# Show the distribution of “conditions_met” separately for PDB vs. DisProt in the TEST SET
if not df_test.empty:
    # Ensure 'label' and 'conditions_met' are in df_test
    if 'label' in df_test.columns and 'conditions_met' in df_test.columns:
        dist_test = df_test.groupby("label")["conditions_met"] \
                      .value_counts() \
                      .unstack(fill_value=0) \
                      .rename(index={0:"DisProt", 1:"PDB"})
        pd.set_option("display.max_columns", None)
        print("Distribution of ‘conditions_met’ by Label (ON TEST SET):\n")
        print(dist_test, "\n")
    else:
        print("Warning: 'label' or 'conditions_met' column missing in df_test for distribution display.")
else:
    print("Warning: Test set is empty. Cannot show distribution of 'conditions_met'.")


# ─── (G) For each k=1…7, classify “folded if conditions_met ≥ k” ON TEST SET ─────────────
results = []
if not df_test.empty and 'conditions_met' in df_test.columns and 'label' in df_test.columns :
    for k_threshold in range(1, 8): # Renamed k to k_threshold to avoid conflict
        # Predictions on the test set
        preds_test = (df_test["conditions_met"] >= k_threshold).astype(int)
        
        # True labels from the test set
        true_test_labels = df_test["label"]
        
        tp = ((preds_test == 1) & (true_test_labels == 1)).sum()
        fn = ((preds_test == 0) & (true_test_labels == 1)).sum()
        tn = ((preds_test == 0) & (true_test_labels == 0)).sum()
        fp = ((preds_test == 1) & (true_test_labels == 0)).sum()
        
        # Calculate accuracy, precision, recall, f1-score for more complete evaluation
        # Avoid division by zero if a class is not present or no predictions for a class
        total_samples_test = len(df_test)
        acc = (tp + tn) / total_samples_test if total_samples_test > 0 else 0
        
        precision_pdb = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall_pdb = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_pdb = 2 * (precision_pdb * recall_pdb) / (precision_pdb + recall_pdb) if (precision_pdb + recall_pdb) > 0 else 0

        precision_disprot = tn / (tn + fn) if (tn + fn) > 0 else 0 # Note: this is not standard precision for class 0
        recall_disprot = tn / (tn + fp) if (tn + fp) > 0 else 0 # Specificity for class 1
        
        results.append({
            "k (min # of features)": k_threshold,
            "TP": tp, "FN": fn, "TN": tn, "FP": fp,
            "Accuracy": f"{acc:.2%}",
            "Precision (PDB)": f"{precision_pdb:.2%}",
            "Recall (PDB)": f"{recall_pdb:.2%}",
            "F1-score (PDB)": f"{f1_pdb:.2%}"
        })

    df_results = pd.DataFrame(results)
    pd.set_option("display.max_rows", None)
    # pd.set_option("display.max_columns", None) # Already set
    print("Performance on TEST SET as we vary k = minimum # of satisfied conditions:\n")
    print(df_results.to_string(index=False))

    # For a more standard report:
    print("\n--- Detailed Classification Report for best k (example: k=4, inspect df_results) ---")
    # Example: Find best k by F1-score for PDB or overall accuracy
    if not df_results.empty:
        # Convert F1-score (PDB) from string percentage to float for finding max
        try:
            df_results['F1_PDB_float'] = df_results['F1-score (PDB)'].str.rstrip('%').astype('float') / 100.0
            best_k_row = df_results.loc[df_results['F1_PDB_float'].idxmax()]
            best_k = int(best_k_row["k (min # of features)"])
            print(f"Best k based on F1-score (PDB) is: {best_k}")
            
            best_preds_test = (df_test["conditions_met"] >= best_k).astype(int)
            print(classification_report(true_test_labels, best_preds_test, target_names=["DisProt (0)", "PDB (1)"]))

            cm = confusion_matrix(true_test_labels, best_preds_test)
            cm_df = pd.DataFrame(cm, index=["Actual DisProt","Actual PDB"], columns=["Pred DisProt","Pred PDB"])
            print("Confusion Matrix for best k:\n", cm_df)

        except KeyError:
            print("Could not determine best k automatically, F1_PDB_float column missing.")
        except Exception as e:
            print(f"Error determining best k: {e}")


else:
    print("Test set is empty or 'conditions_met'/'label' columns are missing. Cannot evaluate performance.")



Loaded 15000 PDB sequences.
Loaded 12787 DisProt sequences.

Training set size: 22229
Testing set size: 5558
Training set PDB (1) count: 12000, DisProt (0) count: 10229
Testing set PDB (1) count: 3000, DisProt (0) count: 2558

Global Feature Means (DisProt vs. PDB) from TRAINING DATA:

         hydro_norm    charge    h_dh_a  norm_flex  pol_norm  arom_plus_helix  \
label                                                                           
DisProt    0.401183 -0.023335  1.255178   0.837046  0.511820         0.719191   
PDB        0.475625 -0.008829  1.071319   0.806036  0.436629         0.734959   

         asa_norm  
label              
DisProt  0.519325  
PDB      0.444460   

Chosen Midpoint Thresholds (from TRAINING DATA):

  hydro_norm         = 0.438
  charge             = -0.016
  h_dh_a             = 1.163
  norm_flex          = 0.822
  pol_norm           = 0.474
  arom_plus_helix    = 0.727
  asa_norm           = 0.482

Distribution of ‘conditions_met’ by Label (ON TEST 